In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext Cython

In [3]:
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
import numpy as np
import scipy.sparse
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics.cluster import adjusted_mutual_info_score

In [5]:
seed = 42
k = 10
n_samples = 10000
random_labels = np.random.RandomState(seed).randint
distribution_a = random_labels(low=0, high=k, size=n_samples)
distribution_b = random_labels(low=0, high=k, size=n_samples)
distribution_b[distribution_b > 5] = 5
C = contingency_matrix(distribution_a, distribution_b, sparse=True)
n_samples = C.sum()
C.shape, n_samples

((10, 6), 10000)

In [6]:
%%cython -a
# cython: cdivision=True
# cython: boundscheck=False
# cython: wraparound=False
#
# Authors: Robert Layton <robertlayton@gmail.com>
#           Corey Lynch <coreylynch9@gmail.com>
# License: BSD 3 clause

from libc.math cimport exp, lgamma
from scipy.special import gammaln
import numpy as np
cimport numpy as np
cimport cython

np.import_array()
ctypedef np.float64_t DOUBLE


def expected_mutual_information(contingency, int n_samples):
    """Calculate the expected mutual information for two labelings."""
    cdef int R, C, a_i, b_j, min_nij, max_nij
    cdef DOUBLE N, gln_N, emi, log_aibj, term2, term3, gln
    cdef np.ndarray[DOUBLE] gln_a, gln_b, gln_Na, gln_Nb, gln_nij, log_Nnij
    cdef np.ndarray[DOUBLE] nijs, term1
    cdef np.ndarray[np.int32_t] a, b, nija
    
    
    R, C = contingency.shape
    N = <DOUBLE>n_samples
    a = np.ravel(contingency.sum(axis=1).astype(np.int32, copy=False))
    b = np.ravel(contingency.sum(axis=0).astype(np.int32, copy=False))
    
    # There are three major terms to the EMI equation, which are multiplied to
    # and then summed over varying nij values.
    # While nijs[0] will never be used, having it simplifies the indexing.
    nijs = np.arange(0, max(np.max(a), np.max(b)) + 1, dtype='float')
    nijs[0] = 1  # Stops divide by zero warnings. As its not used, no issue.
    
    # term1 is nij / N
    term1 = nijs / N

    # term2 uses N * nij
    log_Nnij = np.log(N * nijs)
    
    # term3 is large, and involved many factorials. Calculate these in log
    # space to stop overflows.
    gln_a = gammaln(a + 1)
    gln_b = gammaln(b + 1)
    gln_Na = gammaln(N - a + 1)
    gln_Nb = gammaln(N - b + 1)
    gln_N = gammaln(N + 1)
    gln_nij = gammaln(nijs + 1)
    
    
    # emi itself is a summation over the various values.
    emi = 0
    cdef Py_ssize_t i, j, nij
    for i in range(R):
        a_i = a[i]
        for j in range(C):
            b_j = b[j]
            
            min_nij = int(np.max([1, a_i - N + b_j]))
            max_nij = int(np.min([a_i, b_j]) + 1)
            
            log_aibj = np.log(a_i * b_j)
            for nij in range(min_nij, max_nij):
            
                term2 = log_Nnij[nij] - log_aibj
            
                gln = (
                    gln_a[i]
                    + gln_b[j]
                    + gln_Na[i]
                    + gln_Nb[j]
                    - gln_N
                    - gln_nij[nij]
                    - lgamma(a_i - nij + 1)
                    - lgamma(b_j - nij + 1)
                    - lgamma(N - a_i - b_j + nij + 1)
                )
            
                term3 = exp(gln)
                emi += (term1[nij] * term2 * term3)
    return emi

In [7]:
# should be 0.0022542545598138466
emi = expected_mutual_information(C, n_samples)
emi

0.0022542545598300814

In [8]:
C = scipy.sparse.load_npz('test.npz')
n_samples = C.sum()
n_samples, C.shape

(14975, (5372, 5361))

In [9]:
# should be 6.415867094292364
emi = expected_mutual_information(C, n_samples)
emi

6.4158670938439775